<a href="https://colab.research.google.com/github/Brymer-Meneses/Cat-Dog-Classifier-MobileApplication/blob/master/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Dropout, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, Resizing
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import shutil
import os
from os.path import join
from tqdm.notebook import tqdm
from zipfile import ZipFile

In [2]:
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

--2020-09-23 13:41:16--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.52.161.19, 2600:1400:9000:191::e59, 2600:1400:9000:186::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.52.161.19|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_3367a.zip’

kagglecatsanddogs_3 100%[===================>] 786.68M   105MB/s    in 7.9s    

2020-09-23 13:41:24 (99.6 MB/s) - ‘kagglecatsanddogs_3367a.zip’ saved [824894548/824894548]



In [3]:
path = '/content/kagglecatsanddogs_3367a.zip'
f = ZipFile(path, 'r')
f.extractall()

In [4]:
img_height = 380
img_width = 380
batch_size = 64

In [5]:
ds_dir = '/content/PetImages'

In [6]:
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 1590 images


In [7]:
datagen = ImageDataGenerator(
    validation_split=0.2,
    vertical_flip = True,
    horizontal_flip = True,
    rotation_range=30,
    rescale = 1/255,
    brightness_range = (0.6, 1.6),
    shear_range = 0.05,
    zoom_range = 0.2
)

train_gen = datagen.flow_from_directory(
    ds_dir,
    color_mode = 'rgb',
    class_mode = 'binary',
    subset = 'training',
    shuffle = True,
    batch_size = batch_size,
    
)

val_gen = datagen.flow_from_directory(
    ds_dir,
    color_mode = 'rgb',
    class_mode = 'binary',
    subset = 'validation',
    shuffle = True,
    batch_size = batch_size
)

Found 18729 images belonging to 2 classes.
Found 4681 images belonging to 2 classes.


In [8]:
URL = 'https://tfhub.dev/tensorflow/efficientnet/lite4/feature-vector/2'
pretrained_model = hub.KerasLayer(URL, input_shape = (img_height, img_width, 3))

In [9]:
model = Sequential()
model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(.30))
model.add(Dense(2, activation = 'softmax'))

In [10]:
model.compile(loss = 'sparse_categorical_crossentropy', optimizer =Adam(), metrics = ['accuracy'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              11837936  
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 12,494,834
Trainable params: 656,898
Non-trainable params: 11,837,936
_________________________________________________________________


In [15]:
history = model.fit(train_gen, validation_data = val_gen, epochs = 1)

293/293 [==============================] - 504s 2s/step - loss: 0.1067 - accuracy: 0.9594 - val_loss: 0.0840 - val_accuracy: 0.9680


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
cd /content/drive/My Drive/External Datasets/Models

/content/drive/My Drive/External Datasets/Models


In [18]:
model.save('CvD-EfficientNet-L4-v1.h5')

In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /tmp/tmpirnm0g1q/assets


INFO:tensorflow:Assets written to: /tmp/tmpirnm0g1q/assets


In [21]:
with open('CvD-model-normal.tflite', 'wb') as f:
  f.write(tflite_model)

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpjl25xgpe/assets


INFO:tensorflow:Assets written to: /tmp/tmpjl25xgpe/assets


In [24]:
with open('CvD-model-quantized.tflite', 'wb') as f:
  f.write(tflite_quant_model)